In [1]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Importing the proteins data
ns4b = []
ns4bd = []

f = open("../../../../../../Data/Proteins/DENV2/NS4B/DENV2_NS4B.txt", "r")
for x in f:
    if "DSS" in x:
        ns4bd.append(1)
    elif "DHF" in x:
        ns4bd.append(1)
    elif x[0] == ">":
        ns4bd.append(0)
    else:
        x = x.replace("\n", "")
        ns4b.append(x)

In [3]:
# Converting the arrays into DataFrames
ns4b = pd.DataFrame(ns4b)

# Attaching the "Disease" label column to the input
ns4b["Disease"] = ns4bd

# Renaming the columns
ns4b = ns4b.rename(index=str, columns={0: "Sequence", "Disease": "Disease"})

In [4]:
# Concatinating the input data into one DataFrame
print("The combined shape of the given data is:", str(ns4b.shape))
print("The length of the combined data is:", str(len(ns4b.index)))
print("Does the combined data have any null value? ->", ns4b.isnull().values.any())

# Shuffling the data and then taking a peek
ns4b = ns4b.sample(frac = 1)
print(ns4b.head())

The combined shape of the given data is: (603, 2)
The length of the combined data is: 603
Does the combined data have any null value? -> False
                                              Sequence  Disease
292  NEMGFLEKTKKDLGLGSITTQESESNILDIDLRPASAWTLYAVATT...        0
425  NEMGFLEKTKKDLGLGSITTQESESNILDIDLRPASAWTLYAVATT...        1
111  NEMGFLEKTKKDLGLGSITTQESESNILDIDLRPASAWTLYAVATT...        0
513  NEMGFLEKTKKDLGLGSIATQQPESNILDIDLRPASAWTLYAVATT...        1
57   NEMGFLEKTKKDLGLGSITTQESESNILDIDLRPASAWTLYAVATT...        0


In [5]:
# Function to convert sequence strings into k-mer words, default size = 6 (hexamer words)
def getKmers(sequence, size = 6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

ns4b['words'] = ns4b.apply(lambda x: getKmers(x['Sequence']), axis=1)
ns4b = ns4b.drop('Sequence', axis=1)

ns4b_texts = list(ns4b['words'])
for item in range(len(ns4b_texts)):
    ns4b_texts[item] = ' '.join(ns4b_texts[item])

In [6]:
# Creating y and printing the shape of it
y = ns4b.iloc[:, 0].values
print("The shape of y is:", y.shape)

The shape of y is: (603,)


In [7]:
# Creating the Bag of Words model using CountVectorizer()
# This is equivalent to k-mer counting
# The n-gram size of 4 was previously determined by testing
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(4,4))
x = cv.fit_transform(ns4b_texts)

# Print the shape of x
print("The shape of x matrix is:", x.shape)

The shape of x matrix is: (603, 883)


## Nested Cross-Validation

In [8]:
from sklearn.model_selection import cross_val_score
# search for an optimal value of C for Logistic Regression
Tree_list = [10, 100, 1000, 2500, 5000]
# empty list to store scores
RF_scores = []

In [9]:
from sklearn.ensemble import RandomForestClassifier
# 1. we will loop through reasonable values of k
for trees in Tree_list:
    # 2. run RandomForestClassifier with trees trees
    lr = RandomForestClassifier(n_estimators = trees, criterion = 'entropy', random_state = 0)
    # 3. obtain cross_val_score for RandomForestClassifier with trees trees
    scores = cross_val_score(lr, x, y, cv = 10, scoring='accuracy')
    # 4. append mean of scores for trees trees to RF_scores list
    RF_scores.append(scores.mean())
    
    print("The score for", trees, "trees, is:", scores.mean())

The score for 10 trees, is: 0.749823099008984
The score for 100 trees, is: 0.7497684542002407
The score for 1000 trees, is: 0.7547138093914977
The score for 2500 trees, is: 0.7547138093914977
The score for 5000 trees, is: 0.7547138093914977
